<a href="https://colab.research.google.com/github/LakshmiMedapati96/DM-FinalProject/blob/main/DM_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tushar5harma/topredditcomments

 77% 17.0M/22.1M [00:01<00:00, 25.1MB/s]
100% 22.1M/22.1M [00:01<00:00, 20.1MB/s]


In [ ]:
! unzip topredditcomments.zip

Archive:  topredditcomments.zip
  inflating: Top_Posts.csv           
  inflating: Top_Posts_Comments.csv  


# The Coding begins...

# Importing Libraries, Loading and Merging the data files

In [ ]:
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import string
import re

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Load the dataset
posts = pd.read_csv("Top_Posts.csv")
comments = pd.read_csv("Top_Posts_Comments.csv")

print("posts", posts.shape)
print("comments", comments.shape)

posts (2987, 10)
comments (223174, 2)


In [ ]:
posts.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021


In [ ]:
comments.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...


In [ ]:
posts.subreddit.value_counts()

artificial         999
MachineLearning    998
datascience        990
Name: subreddit, dtype: int64

In [ ]:
# The code in this cell was written by me by referring to Reference [1].

# Merging the 2 input files by using the common column "post_id" in both the files
merged_data = pd.merge(posts, comments, on="post_id", how="inner")

merged_data

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year,comment
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,The future 🤯
2,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Ohh the nightmare of making this into a stable...
...,...,...,...,...,...,...,...,...,...,...,...
223163,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,LiDAR is mot powerful sensor for the auto driv...
223164,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,So it can now idenrify traffic lights? Musk pr...
223165,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,Hydranet bro!
223166,efk5n3,Tesla's Neural Net can now identify red and gr...,artificial,https://www.teslarati.com/tesla-holiday-update...,NaN,80,10,0.89,2019-12-25 18:50:50,2019,It even shows flashing yellow turn arrows.


In [ ]:
# Storing the necessary columns only
merged_data = merged_data[["comment", "subreddit"]]

merged_data.head()

,comment,subreddit
0,Twitter thread: [https://twitter.com/cyrildiag...,MachineLearning
1,The future 🤯,MachineLearning
2,Simple yet very useful. Thank you for sharing ...,MachineLearning
3,"Almost guaranteed, Apple will copy your idea i...",MachineLearning
4,Ohh the nightmare of making this into a stable...,MachineLearning


# Data Preprocessing

In [ ]:
print(merged_data.shape)

# checking for missing values
merged_data.isnull().sum()


(223168, 2)


comment      9
subreddit    0
dtype: int64

In [ ]:
# Dropping the missing values
merged_data.dropna(inplace=True)

print(merged_data.shape)

(223159, 2)


<ipython-input-44-2b16886c28c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.dropna(inplace=True)


In [ ]:
# The below code is taken from Reference [2].
# Defining a function to remove URLs
def remove_urls(doc):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', doc)


# The below code is taken from Reference [3].
# Defining a function to remove emoji
def remove_emoji(doc):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"          # emoticons
        u"\U0001F300-\U0001F5FF"          # symbols & pictographs
        u"\U0001F680-\U0001F6FF"          # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"          # flags (iOS)
        u"\U00002500-\U00002BEF"          # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"                         # dingbats
        u"\u3030"
        "]+", re.UNICODE)
    return emoji_pattern.sub(r'', doc)


# Written by me by referring to Reference [4]. 
stop_words = stopwords.words('english')           # Accessing stopwords from nltk library

# Written by me by referring to Reference [5].
lemmatizer = WordNetLemmatizer()                  # Lemmatization


# Below lines of code are my own code
# Defining a function to clean the data
def clean(doc):
    doc = str(doc)                                              # converting to string 
    doc = doc.lower().strip()                                   # converting to lower case and stripping the whitespaces
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])          # removing punctuations and digits

  # Written by me by referring to Reference [4].
    doc = " ".join([token for token in doc.split() if token not in stop_words])                 # splitting the doc into tokens and removing stopwords
  
  # Written by me by referring to Reference [5].
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])                                 # lemmatizing the doc
    return doc

In [ ]:
# Cleaning the dataset
merged_data["comment"] = merged_data["comment"].apply(remove_urls)  
merged_data["comment"] = merged_data["comment"].apply(remove_emoji)          
merged_data["comment"] = merged_data["comment"].apply(clean)          

<ipython-input-46-af3d8f411adc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data["comment"] = merged_data["comment"].apply(remove_urls)
<ipython-input-46-af3d8f411adc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data["comment"] = merged_data["comment"].apply(remove_emoji)
<ipython-input-46-af3d8f411adc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [ ]:
merged_data.head()

,comment,subreddit
0,twitter thread code background removal done un...,MachineLearning
1,future,MachineLearning
2,simple yet useful thank sharing code,MachineLearning
3,almost guaranteed apple copy idea,MachineLearning
4,ohh nightmare making stable product enough dri...,MachineLearning


In [ ]:
# Splitting the dataset into Train, Validation and Test sets - 60:20:20 ratio
X = merged_data['comment']
y = merged_data['subreddit']

# Split the data into training and remaining datasets
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size = 0.6)

# Split the remaining dataset into validation and test datasets
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, train_size = 0.5)


print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(133895,) (133895,)
(44632,) (44632,)
(44632,) (44632,)


In [ ]:
# Written by me by referring to Reference [6].
# Vectorize dataset 
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

# **Model 1 :** Decision Tree Classifier

In [ ]:
# Written by me by referring to Reference [6].

dtc = DecisionTreeClassifier(max_depth = 3)
dtc = dtc.fit(X_train, y_train)

y_pred_train = dtc.predict(X_train)
y_pred_val = dtc.predict(X_val) 
y_pred_test = dtc.predict(X_test)

accuracy_val_DT_BHT = accuracy_score(y_val, y_pred_val)
accuracy_test_DT_BHT = accuracy_score(y_test, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_DT_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_DT_BHT)

Accuracy on validation set before hyperparameter tuning : 0.5266176734181753
Accuracy on test set before hyperparameter tuning: 0.5218005018820577


Hyperparameter Tuning with GridSearchCV

In [ ]:
# Written by me by referring to Reference [7] and [8].

param_dict = {
    "max_depth": range(13, 15),
    "min_samples_split": [4, 5, 6]
}

grid_search_dt = GridSearchCV(dtc, param_dict, cv = 10)

grid_search_dt.fit(X_train, y_train)
print("Best hyperparameters:", grid_search_dt.best_params_)


accuracy_val_DT_AHT = grid_search_dt.best_estimator_.score(X_val, y_val)
accuracy_test_DT_AHT = grid_search_dt.best_estimator_.score(X_test, y_test)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_DT_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_DT_AHT)

Best hyperparameters: {'max_depth': 14, 'min_samples_split': 6}
Accuracy on validation set after hyperparameter tuning :  0.5606067395590608
Final accuracy on test set after hyperparameter tuning :  0.5577836529844058


In [ ]:
prediction = grid_search_dt.best_estimator_(X_test)
print(classification_report(y_test, prediction, target_names=['MachineLearning', 'datascience','artificial']))

TypeError: ignored

# **Model 2 :** Random Forest Classifier

In [ ]:
# Written by me by referring to Reference [9].

rfc = RandomForestClassifier(n_estimators = 110, max_depth = 2)
rfc = rfc.fit(X_train, y_train)

y_pred_train = rfc.predict(X_train)
y_pred_val = rfc.predict(X_val) 
y_pred_test = rfc.predict(X_test)

accuracy_val_RF_BHT = accuracy_score(y_val, y_pred_val)
accuracy_test_RF_BHT = accuracy_score(y_test, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_RF_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_RF_BHT)

Accuracy on validation set before hyperparameter tuning : 0.47761695644380714
Accuracy on test set before hyperparameter tuning: 0.4838456712672522


Hyperparameter Tuning with GridSearchCV

In [ ]:
param_dict = {
    "n_estimators" : [25, 50, 100],
    "max_depth": range(13, 15)  
}

grid_search_rf = GridSearchCV(rfc, param_dict, cv = 10)

grid_search_rf.fit(X_train, y_train)
print("Best hyperparameters:", grid_search_rf.best_params_)


accuracy_val_RF_AHT = grid_search_rf.best_estimator_.score(X_val, y_val)
accuracy_test_RF_AHT = grid_search_rf.best_estimator_.score(X_test, y_test)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_RF_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_RF_AHT)

Best hyperparameters: {'max_depth': 14, 'n_estimators': 100}
Accuracy on validation set after hyperparameter tuning :  0.5686951066499373
Final accuracy on test set after hyperparameter tuning :  0.5721455457967378


# **Model 3 :** k Nearest Neighbours

In [ ]:
# Written by me by referring to Reference [10].

knc = KNeighborsClassifier(n_neighbors = 3)
knc = knc.fit(X_train, y_train)

y_pred_train = knc.predict(X_train)
y_pred_val = knc.predict(X_val) 
y_pred_test = knc.predict(X_test)

accuracy_val_KNN_BHT = accuracy_score(y_val, y_pred_val)
accuracy_test_KNN_BHT = accuracy_score(y_test, y_pred_test)
print("Accuracy on validation set before hyperparameter tuning :",accuracy_val_KNN_BHT)
print("Accuracy on test set before hyperparameter tuning:",accuracy_test_KNN_BHT)

Accuracy on validation set before hyperparameter tuning : 0.5451469797454741
Accuracy on test set before hyperparameter tuning: 0.5457295214196093


Hyperparameter Tuning with GridSearchCV

In [ ]:
param_dict = {
    "n_neighbors" : [5, 7, 11]  
}

grid_search_kn = GridSearchCV(knc, param_dict, cv = 10)

grid_search_kn.fit(X_train, y_train)
print("Best hyperparameters:", grid_search_kn.best_params_)


accuracy_val_KNN_AHT = grid_search_kn.best_estimator_.score(X_val, y_val)
accuracy_test_KNN_AHT = grid_search_kn.best_estimator_.score(X_test, y_test)
print("Accuracy on validation set after hyperparameter tuning : ", accuracy_val_KNN_AHT)
print("Final accuracy on test set after hyperparameter tuning : ", accuracy_test_KNN_AHT)

Best hyperparameters: {'n_neighbors': 11}
Accuracy on validation set after hyperparameter tuning :  0.5544676465316365
Final accuracy on test set after hyperparameter tuning :  0.5615029575192687


# Unnecessary

In [ ]:
# gnb = GaussianNB() 
# %time gnb.fit(X_train, y_train)

# y_pred_train = gnb.predict(X_train)
# y_pred_val = gnb.predict(X_val)
# y_pred_test = gnb.predict(X_test)

# print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
# print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))
# print("Validation Accuracy score:",accuracy_score(y_val, y_pred_val))

In [ ]:
# print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))

# Using MultiNormial Naive Bayes

In [ ]:
mnb = MultinomialNB() 
%time mnb.fit(vocab_list_trainset, y_train)

y_pred_train = mnb.predict(vocab_list_trainset)
y_pred_test = mnb.predict(X_test)
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


In [ ]:
print(classification_report(y_test, y_pred_test, target_names=['MachineLearning', 'datascience','artificial']))